In [15]:
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 전처리
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import random

# time
from datetime import datetime , timedelta

# 데이터 프레임 옵션
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action = "ignore", category = SettingWithCopyWarning)

# 시각화
import matplotlib.pyplot as plt
%matplotlib inline 

# modeling
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,recall_score, precision_score, roc_auc_score, f1_score ,classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
import tensorflow as tf
import keras
from keras import utils
from keras import layers, models
from keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [2]:
df_final = pd.read_csv('./data/df_final.csv')
df_final

,user_f_id,period,class_buy_counts,101_counts,page_view_counts,page_view_kinds,wishlist_count,reward_counts,crm_click_counts,coupon_download_counts,type
0,0Hoa5DPMmSV7oKs0WuDiucgjU5E3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,active
1,xBpgoQ3NwKZ8jLWML1SY4nPrGWq1,12,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,active
2,kakao1596934947,37,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,active
3,kakao1503416525,109,1.0,0.0,13.0,10.0,1.0,0.0,0.0,0.0,active
4,kakao1578462319,32,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,active
...,...,...,...,...,...,...,...,...,...,...,...
1216089,kakao1436506038,133,0.0,0.0,13.0,5.0,2.0,0.0,0.0,0.0,dormant
1216090,kakao1595347311,1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,dormant
1216091,kakao1454120650,1,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,dormant
1216092,kakao1499983752,1,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,dormant


# 11. 예측값 라벨링(Y) 추가
- utm x, buy x : 0
- utm o, buy x : 1
- utm o, buy o : 2

In [3]:
df_final_crm_click_counts = list(df_final['crm_click_counts'].map(lambda x: 1 if x>0 else 0))
df_final_coupon_download_counts = list(df_final['coupon_download_counts'].map(lambda x: 1 if x>0 else 0))

In [4]:
df_final['df_final_crm_click_counts'] = df_final_crm_click_counts
df_final['df_final_coupon_download_counts'] = df_final_coupon_download_counts

In [5]:
df_final['label'] = df_final['df_final_crm_click_counts'] + df_final['df_final_coupon_download_counts']
df_final['label']

0          0
1          0
2          0
3          0
4          0
          ..
1216089    0
1216090    0
1216091    0
1216092    0
1216093    0
Name: label, Length: 1216094, dtype: int64

In [6]:
# df_final['label'] = [1 if df_final['df_final_crm_click_counts'][i] + df_final['df_final_coupon_download_counts'][i] > 0 else 0 for i in range(len(df_final))]
# df_final['label'].value_counts()

In [7]:
df_active = df_final[df_final['type'] == 'active']
df_dormant = df_final[df_final['type'] == 'dormant']

In [8]:
df_active_final = df_active[['period', 'class_buy_counts', '101_counts','page_view_counts', 'page_view_kinds', 'wishlist_count', 'reward_counts', 'label']]
df_active_final = df_active_final.astype('float64')
df_active_final

,period,class_buy_counts,101_counts,page_view_counts,page_view_kinds,wishlist_count,reward_counts,label
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0
2,37.0,0.0,0.0,6.0,3.0,0.0,0.0,0.0
3,109.0,1.0,0.0,13.0,10.0,1.0,0.0,0.0
4,32.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
426677,26.0,1.0,0.0,21.0,1.0,1.0,0.0,1.0
426678,13.0,0.0,0.0,10.0,4.0,0.0,0.0,0.0
426679,94.0,3.0,0.0,48.0,23.0,2.0,0.0,0.0
426680,141.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0


In [9]:
# train test set
X = df_active_final[['period', 'class_buy_counts', '101_counts', 'page_view_counts','page_view_kinds', 'wishlist_count', 'reward_counts']].values
y = df_active_final['label'].values.ravel()

# scaling
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# oversampling
smote = SMOTE(random_state = 11)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.2, shuffle = True, random_state = 1) 

In [10]:
# y 값 원핫인코딩

y_train_final = to_categorical(y_train)
y_test_final = to_categorical(y_test)

In [19]:
# 최종 학습 모델
model = load_model('final_model.h5')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 15)                120       
_________________________________________________________________
dense_17 (Dense)             (None, 30)                480       
_________________________________________________________________
dense_18 (Dense)             (None, 15)                465       
_________________________________________________________________
dense_19 (Dense)             (None, 3)                 48        
Total params: 1,113
Trainable params: 1,113
Non-trainable params: 0
_________________________________________________________________


In [12]:
# DNN 모델링

# model = models.Sequential()
# model.add(layers.Dense(30,activation='elu',input_shape=(7,)))
# model.add(layers.Dense(60,activation='elu'))
# model.add(layers.Dense(30,activation='elu'))
# model.add(layers.Dense(3,activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# model.summary()

In [17]:
model.fit(X_train, y_train_final, epochs = 100, batch_size = 500)
prediction = model.predict_classes(X_test)
print(classification_report(y_test, prediction))

Epoch 1/100
1842/1842 [==============================] - 2s 750us/step - loss: 0.6730 - accuracy: 0.6892
Epoch 2/100
1842/1842 [==============================] - 1s 773us/step - loss: 0.6589 - accuracy: 0.6947
Epoch 3/100
1842/1842 [==============================] - 1s 742us/step - loss: 0.6541 - accuracy: 0.6971
Epoch 4/100
1842/1842 [==============================] - 1s 761us/step - loss: 0.6512 - accuracy: 0.6994
Epoch 5/100
1842/1842 [==============================] - 1s 760us/step - loss: 0.6481 - accuracy: 0.7018
Epoch 6/100
1842/1842 [==============================] - 1s 769us/step - loss: 0.6460 - accuracy: 0.7034
Epoch 7/100
1842/1842 [==============================] - 2s 829us/step - loss: 0.6443 - accuracy: 0.7045
Epoch 8/100
1842/1842 [==============================] - 1s 766us/step - loss: 0.6429 - accuracy: 0.7050
Epoch 9/100
1842/1842 [==============================] - 1s 791us/step - loss: 0.6416 - accuracy: 0.7061
Epoch 10/100
1842/1842 [==============================]

In [40]:
# dormant 유저 확인

df_dormant_final = df_dormant[['period', 'class_buy_counts', '101_counts', 'page_view_counts','page_view_kinds', 'wishlist_count', 'reward_counts']].values
df_dormant_final

array([[150.,   0.,   0., ...,  19.,   5.,   0.],
       [ 28.,   0.,   0., ...,   1.,   0.,   0.],
       [  1.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  1.,   0.,   1., ...,   1.,   0.,   0.],
       [  1.,   0.,   0., ...,   2.,   0.,   0.],
       [  1.,   0.,   0., ...,   1.,   0.,   1.]])

In [45]:
prediction = model.predict_classes(df_dormant_final)

In [46]:
dd = pd.DataFrame(prediction, columns = ['prediction'])
df_dormant.reset_index(drop=True, inplace=True)
df_dormant['prediction'] = dd['prediction']
final = df_dormant[['user_f_id', 'prediction']]
final

,user_f_id,prediction
0,kakao1467463783,2
1,kakao1558323581,0
2,txbE3PMKWUbTVf9JXhtP521QDt82,0
3,kakao1476609662,1
4,kakao1581415551,1
...,...,...
789407,kakao1436506038,0
789408,kakao1595347311,1
789409,kakao1454120650,1
789410,kakao1499983752,1
